# MSCOCO 한국어 데이터

In [ ]:
from konlpy.tag import Mecab
from collections import defaultdict
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer,TfidfVectorizer
import random
import pandas as pd
import os, glob
import os.path
import import_ipynb 
import love_poem_crawler, oldpoem_crawling, poemcorpus
import re

In [ ]:
df = pd.read_json('MSCOCO_train_val_Korean.json')
df[:5]

In [ ]:
mscoco =  df['caption_ko']
mscoco[:5]

In [ ]:
import random

In [ ]:
random.randint(0,1)

In [ ]:
mecab = Mecab()
preprocessed_docs = []
example_mscoco = []
for i in mscoco:
    token_list = []
    for token in mecab.pos(i[random.randint(0,len(i)-1)]):
        if token[1] in ['NNG','VA+ETM']:
            token_list.append(token[0])
    preprocessed_docs.append(" ".join(token_list))
    example_mscoco.append(token_list)
display(preprocessed_docs[:10],len(preprocessed_docs),example_mscoco[:5])


In [ ]:
vectorizer = TfidfVectorizer()
sp_matrix = vectorizer.fit_transform(preprocessed_docs)
word2id = defaultdict(lambda : 0)
for idx, feature in enumerate(vectorizer.get_feature_names()):
    word2id[feature] = idx
keywords_mscoco = []
for i, sent in enumerate(preprocessed_docs):
    print('====== document[%d] ======' % i)
    temp = []
    for token in sent.split():
        matrix = sp_matrix[i, word2id[token]]
        print(token,matrix)
        if 0.3 <= matrix < 1.0:
            temp.append(token)
    if len(temp)==0:
        keywords_mscoco.append('none')
    else:
        keywords_mscoco.append(",".join(temp))
display(keywords_mscoco[:5],len(keywords_mscoco))

In [ ]:
df['keywords'] = keywords_mscoco

In [ ]:
df.to_csv('mscoco_modified.csv', encoding='utf-8')

In [ ]:
import pandas as pd
from tqdm import tqdm

In [ ]:
data = pd.read_csv('mscoco_modified.csv')
ko_np = pd.read_csv('korean_nonpup_lines.csv')
del ko_np['Unnamed: 0']
del data['Unnamed: 0']

In [ ]:
data[:1]

In [ ]:
ko_np[:1]

In [ ]:
keywords = list(data['keywords'])

In [ ]:
ko_nonpup = []

for words in tqdm(keywords):
    count = 0
    
    for word in words.split(','):
        for sent in ko_np['0']:
            if word in sent and sent not in ko_nonpup:
                ko_nonpup.append(sent)
                count+=1
                break
        if count == 1:
            break
    if count == 0:
        ko_nonpup.append('none')

In [ ]:
print('data: {}, ko_nonpup: {}'.format(len(data), len(ko_nonpup)))

In [ ]:
ko_nonpup.count('none')

In [ ]:
data['ko_nonpup'] = ko_nonpup
data.to_csv('mscoco_ko_nonpup.csv', encoding='utf-8')

In [ ]:
data[:5]

In [ ]:
import random 
combined = []
for cap, ko_non in tqdm(zip(data['caption_ko'], data['ko_nonpup'])):
    if ko_non == 'none':
        combined.append(re.sub('\[|\]|\'|\"',"",cap).split(', ')[random.randint(0,len(re.sub('\[|\]|\'|\"',"",cap).split(', '))-1)])
    else:
        combined.append(ko_non)

In [ ]:
data['combined'] = combined

In [ ]:
data.to_csv('mscoco_kononpup_combined.csv', encoding='utf-8')